ctro = round.([ x_min + espacio[1]/2, y_min + espacio[2]/2,  z_min + espacio[3]/2 ], digits = 3)

dim = ncells * cutoff

half_dim = round(dim / 2, digits = 3)

sa = contacto.Voxel(ctro, half_dim, 0)

In [2]:
using Revise
using contacto
using Chemfiles
using StaticArrays

In [3]:
in_trj = Trajectory("/home/german/labo/contacto/aux/1mtn.pdb")
in_frm = read(in_trj)
in_top = Topology(in_frm)
coords = round.(positions(in_frm), digits = 3)
natoms = size(coords)[2]

#############

cutoff = round(5., digits = 3)
resolution = .1
t_xyz = transpose(coords)

srt_x_idx_t_xyz = sortperm(t_xyz[:, 1])
srt_x_t_xyz = t_xyz[srt_x_idx_t_xyz, 1]

x_min = floor(srt_x_t_xyz[1])
x_max = ceil(srt_x_t_xyz[end])
y_min = floor(minimum(t_xyz[:, 2]))
y_max = ceil(maximum(t_xyz[:, 2]))
z_min = floor(minimum(t_xyz[:, 3]))
z_max = ceil(maximum(t_xyz[:, 3]))

espacio = [ x_max - x_min ; y_max - y_min ; z_max - z_min ]
ncells = convert(Int64, ceil(maximum(espacio) / cutoff))
box_dim = [ x_min, x_max, y_min, y_max, z_min, z_max ]
box_vtces = hcat(
    [ x_min ; x_max ; x_min ; x_max ; x_min ; x_max ; x_min ; x_max ], 
    [ y_min ; y_min ; y_max ; y_max ; y_min ; y_min ; y_max ; y_max ],
    [ z_min ; z_min ; z_min ; z_min ; z_max ; z_max ; z_max ; z_max ])
#############

init_vox = vcat([ x_min y_min z_min ], [ x_min+cutoff y_min z_min ],
    [ x_min y_min+cutoff z_min ], [ x_min+cutoff y_min+cutoff z_min ],
    [ x_min y_min z_min+cutoff ], [ x_min+cutoff y_min z_min+cutoff ],
    [ x_min y_min+cutoff z_min+cutoff ], [ x_min+cutoff y_min+cutoff z_min+cutoff ])

celdas = Array{Array{Float64, 2}, 1}(undef, ncells^3);
let cell_vtces = init_vox, m = 1, vect_z = zeros(8)
    for i = 1:ncells
        let vect_y = zeros(8)
            for j = 1:ncells
                let vect_x = zeros(8)
                    for k = 1:ncells
                        global celdas[m] = cell_vtces + hcat(vect_x, vect_y, vect_z)
                        m += 1
                        vect_x .+= cutoff
                    end
                end
                vect_y .+= cutoff
            end
        end
        vect_z .+= cutoff
    end
end

#############
nlevels = convert(Int64, ceil(log2(ncells)))
nnodes = sum([ 2^x for x in 1:(nlevels) ]) ; sobra = (2^nlevels - ncells) * 2 ; nnodes -= sobra

nodes_anchos = Array{Array{Int64, 1}, 1}(undef, nlevels+1)
nodes_anchos[1] = [ncells]
nodes_ranges = Array{Array{Tuple{Int64,Int64}, 1}, 1}(undef, nlevels+1)
nodes_ranges[1] = [(0, ncells)]
nodes_bounds = Array{Array{Int64, 1}, 1}(undef, nlevels+1)
nodes_bounds[1] = [ncells]

let i = 1, j = 1; for i = 1:nlevels
    k = i + 1
    nodes_anchos[k] = []
    nodes_ranges[k] = []
    nodes_bounds[k] = []
    prev = 0
    for j = 1:2^(i-1)
        
        lh_node_ancho = convert(Int64, ceil(nodes_anchos[i][j] / 2))
        rh_node_ancho = nodes_anchos[i][j] - lh_node_ancho
        push!(nodes_anchos[k], lh_node_ancho)
        push!(nodes_anchos[k], rh_node_ancho)
        
        rango = nodes_ranges[i][j][2] - nodes_ranges[i][j][1]
        lh_node_rango = convert(Int64, ceil(rango / 2)) + prev
        rh_node_rango = rango + prev
        push!(nodes_ranges[k], (prev, lh_node_rango))
        push!(nodes_ranges[k], (lh_node_rango, rh_node_rango))
            
        push!(nodes_bounds[k], lh_node_rango)
        prev = rh_node_rango
    end
end
end

┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:35 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:35
┌ Warning: `assert` is deprecated, use `@assert` instead.
│   caller = read(::Trajectory) at Trajectory.jl:36
└ @ Chemfiles /home/german/.julia/packages/Chemfiles/MzvVI/src/Trajectory.jl:36
┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:108 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:108
┌ Warning: `assert` is deprecated, use `@assert` instead.
│   caller = ip:0x0
└ @ Core :-1
┌ Warning: `finalizer(o, f::Function)` is deprecated, use `finalizer(f, o)` instead.
│   caller = Type at Chemfiles.jl:51 [inlined]
└ @ Core /home/german/.julia/packages/Chemfiles/MzvVI/src/Chemfiles.jl:51
┌ Warning: `unsafe_wrap(T, pointer, dims, own)` is deprecated, use `unsafe_wrap(T, pointer, dims, o

In [4]:
nodes_anchos

5-element Array{Array{Int64,1},1}:
 [11]                                            
 [6, 5]                                          
 [3, 3, 3, 2]                                    
 [2, 1, 2, 1, 2, 1, 1, 1]                        
 [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0]

In [5]:
nodes_ranges

5-element Array{Array{Tuple{Int64,Int64},1},1}:
 [(0, 11)]                                                                                                                              
 [(0, 6), (6, 11)]                                                                                                                      
 [(0, 3), (3, 6), (6, 9), (9, 11)]                                                                                                      
 [(0, 2), (2, 3), (3, 5), (5, 6), (6, 8), (8, 9), (9, 10), (10, 11)]                                                                    
 [(0, 1), (1, 2), (2, 3), (3, 3), (3, 4), (4, 5), (5, 6), (6, 6), (6, 7), (7, 8), (8, 9), (9, 9), (9, 10), (10, 10), (10, 11), (11, 11)]

In [6]:
nodes_bounds

5-element Array{Array{Int64,1},1}:
 [11]                      
 [6]                       
 [3, 9]                    
 [2, 5, 8, 10]             
 [1, 3, 4, 6, 7, 9, 10, 11]

In [7]:
nodes_indices_x = Array{Array{UnitRange{Int64}, 1}, 1}(undef, nlevels+1)
nodes_indices_x[1] = [1:natoms]
for i = 1:nlevels
    k = i + 1
    nodes_indices_x[k] = []
    for j = 1:2^(i-1)
        englobing_range = nodes_indices_x[i][j]
        ancho = nodes_ranges[k][j*2][2] - nodes_ranges[k][j*2-1][1]
        if ancho > 1
            if length(englobing_range) > 1
                x_idx = searchsortedlast(srt_x_t_xyz[englobing_range],
                    (x_min + cutoff * nodes_bounds[k][j])) 
        
                if x_idx == 0
                    # Todos los átomos en la celda de la derecha
                    push!(nodes_indices_x[k], 0:0)
                    push!(nodes_indices_x[k], englobing_range)
                elseif x_idx == length(englobing_range)
                    # Todos los átomos en la celda de la izquierda
                    push!(nodes_indices_x[k], englobing_range)
                    push!(nodes_indices_x[k], 0:0)
                else
                    # Divido en 2 celdas más 
                    x_boundary = x_idx + englobing_range.start - 1
                    push!(nodes_indices_x[k],
                        nodes_indices_x[i][j].start:x_boundary)
                    push!(nodes_indices_x[k],
                        x_boundary+1:nodes_indices_x[i][j].stop)
                end
            elseif englobing_range.start == 0 & englobing_range.stop == 0
                # 0 átomos en la celda
                push!(nodes_indices_x[k], 0:0)
                push!(nodes_indices_x[k], 0:0)
            else
                error("re loco.")
            end
        else
            push!(nodes_indices_x[k], englobing_range)
        end
    end
end

In [8]:
pl = 1

#if !(nodes_indices_x[end][pl].start == 0 & nodes_indices_x[end][pl].start == 0)

t_xyz_plano = t_xyz[srt_x_idx_t_xyz[nodes_indices_x[end][pl]], :]
srt_y_idx_t_xyz_plano = sortperm(t_xyz_plano[:, 2])
srt_y_t_xyz_plano = t_xyz_plano[srt_y_idx_t_xyz_plano, 2]

nlevels
ncells
nnodes
natoms_plano = length(nodes_indices_x[end][pl])

12

In [9]:
nodes_indices_y_plano = Array{Array{UnitRange{Int64}, 1}, 1}(undef, nlevels+1)
nodes_indices_y_plano[1] = [1:natoms_plano]

for i = 1:nlevels
    k = i + 1
    nodes_indices_y_plano[k] = []
    for j = 1:2^(i-1)
        englobing_range = nodes_indices_y_plano[i][j]
        ancho = nodes_ranges[k][j*2][2] - nodes_ranges[k][j*2-1][1]
        if ancho > 1 
            if length(englobing_range) > 1
                y_idx = searchsortedlast(
                    srt_y_t_xyz_plano[englobing_range],
                    (y_min + cutoff * nodes_bounds[k][j])) 
                
                if y_idx == 0
                    # Todos los átomos en la celda de la derecha
                    push!(nodes_indices_y_plano[k], 0:0)
                    push!(nodes_indices_y_plano[k], englobing_range)
                elseif y_idx == length(englobing_range)
                    # Todos los átomos en la celda de la izquierda
                    push!(nodes_indices_y_plano[k], englobing_range)
                    push!(nodes_indices_y_plano[k], 0:0)
                else
                    # Divido en 2 celdas más 
                    y_boundary = y_idx + englobing_range.start - 1
                    push!(nodes_indices_y_plano[k],
                        nodes_indices_y_plano[i][j].start:y_boundary - 1)
                    push!(nodes_indices_y_plano[k],
                        y_boundary:nodes_indices_y_plano[i][j].stop)
                end
            elseif englobing_range.start == 0 & englobing_range.stop == 0
                # 0 átomos en la celda
                push!(nodes_indices_y_plano[k], 0:0)
                push!(nodes_indices_y_plano[k], 0:0)
            else
                error("re loco.")
            end
        else
            push!(nodes_indices_y_plano[k], englobing_range)
        end
    end
end

In [59]:
eje = 1

#if !(nodes_indices_x[end][eje].start == 0 & nodes_indices_x[end][eje].start == 0)

t_xyz_eje = t_xyz[srt_x_idx_t_xyz[nodes_indices_y_plano[end][eje]], :]
srt_z_idx_t_xyz_eje = sortperm(t_xyz_eje[:, ]) 
srt_z_t_xyz_eje = t_xyz_eje[srt_z_idx_t_xyz_eje, 3]

nlevels
ncells
nnodes
natoms_eje = length(nodes_indices_y_plano[end][eje])

BoundsError: BoundsError: attempt to access 1989-element Array{Int64,1} at index [0:0]

In [10]:
nodes_indices_x[end]

11-element Array{UnitRange{Int64},1}:
 1:12     
 13:50    
 51:118   
 119:247  
 248:437  
 438:699  
 700:1046 
 1047:1411
 1412:1691
 1692:1908
 1909:1989

In [11]:
nodes_indices_y_plano[end]

11-element Array{UnitRange{Int64},1}:
 0:0 
 0:0 
 1:4 
 5:12
 0:0 
 0:0 
 0:0 
 0:0 
 0:0 
 0:0 
 0:0 

# Este será el código posta

In [219]:
in_trj = Trajectory("/home/german/labo/contacto/aux/1mtn.pdb")
in_frm = read(in_trj)
in_top = Topology(in_frm)
coords = positions(in_frm) 
cutoff = round(10., digits = 3)
resolution = .1

###################

xyz = round.(transpose(coords), digits = 3)
natoms = size(xyz)[1]

x_min = floor(minimum(xyz[:, 1]))
x_max = ceil(maximum(xyz[:, 1]))
y_min = floor(minimum(xyz[:, 2]))
y_max = ceil(maximum(xyz[:, 2]))
z_min = floor(minimum(xyz[:, 3]))
z_max = ceil(maximum(xyz[:, 3]))

espacio = [ x_max - x_min ; y_max - y_min ; z_max - z_min ]
ncells = convert(Int64, ceil(maximum(espacio) / cutoff))
box_dim = [ x_min, x_max, y_min, y_max, z_min, z_max ]
xyz_min = [ x_min, y_min, z_min ]

box_vtces = hcat(
    [ x_min ; x_max ; x_min ; x_max ; x_min ; x_max ; x_min ; x_max ], 
    [ y_min ; y_min ; y_max ; y_max ; y_min ; y_min ; y_max ; y_max ],
    [ z_min ; z_min ; z_min ; z_min ; z_max ; z_max ; z_max ; z_max ])

init_vox = vcat([ x_min y_min z_min ], [ x_min+cutoff y_min z_min ],
    [ x_min y_min+cutoff z_min ], [ x_min+cutoff y_min+cutoff z_min ],
    [ x_min y_min z_min+cutoff ], [ x_min+cutoff y_min z_min+cutoff ],
    [ x_min y_min+cutoff z_min+cutoff ], [ x_min+cutoff y_min+cutoff z_min+cutoff ])

celdas = Array{Array{Float64, 2}, 1}(undef, ncells^3);
let cell_vtces = init_vox, m = 1, vect_z = zeros(8)
    for i = 1:ncells
        let vect_y = zeros(8)
            for j = 1:ncells
                let vect_x = zeros(8)
                    for k = 1:ncells
                        global celdas[m] = cell_vtces + hcat(vect_x, vect_y, vect_z)
                        m += 1
                        vect_x .+= cutoff
                    end
                end
                vect_y .+= cutoff
            end
        end
        vect_z .+= cutoff
    end
end

nlevels = convert(Int64, ceil(log2(ncells)))
nodes_ranges = Array{Array{Tuple{Int64,Int64}, 1}, 1}(undef, nlevels+1)
nodes_ranges[1] = [(0, ncells)]
nodes_bounds = Array{Array{Int64, 1}, 1}(undef, nlevels+1)
nodes_bounds[1] = [ncells]

let i = 1, j = 1; for i = 1:nlevels
    k = i + 1
    nodes_ranges[k] = []
    nodes_bounds[k] = []
    prev = 0
    for j = 1:2^(i-1)
        rango = nodes_ranges[i][j][2] - nodes_ranges[i][j][1]
        lh_node_rango = convert(Int64, ceil(rango / 2)) + prev
        rh_node_rango = rango + prev
        push!(nodes_ranges[k], (prev, lh_node_rango))
        push!(nodes_ranges[k], (lh_node_rango, rh_node_rango))
            
        push!(nodes_bounds[k], lh_node_rango)
        prev = rh_node_rango
    end
end
end

┌ Warning: Deprecated syntax `implicit assignment to global variable `i``.
│ Use `global i` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `j``.
│ Use `global j` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `k``.
│ Use `global k` instead.
└ @ nothing none:0
┌ Warning: Loop variable `i` around In[219]:37 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[219]:37
┌ Warning: Loop variable `j` around In[219]:39 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[219]:39
┌ Warning: Loop variable `k` around In[219]:41 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[219]:41


nodes_indices = Array{Array{Array{UnitRange{Int64}, 1}, 1}, 1}(undef, 3)
arbol = Array{Array{Array{UnitRange{Int64}, 1}, 1}, 1}(undef, 4)
srt_idx_tree = Array{Array{Array{Int64, 1}, 1}, 1}(undef, 4)

arbol[1] = [[1:natoms]]
arbol[2] = Array{Array{UnitRange{Int64}, 1}, 1}(undef, 1)
arbol[3] = Array{Array{UnitRange{Int64}, 1}, 1}(undef, ncells)
arbol[4] = Array{Array{UnitRange{Int64}, 1}, 1}(undef, ncells^2)

srt_idx_tree[2] = Array{Array{Int64, 1}, 1}(undef, 1)
srt_idx_tree[3] = Array{Array{Int64, 1}, 1}(undef, ncells)
srt_idx_tree[4] = Array{Array{Int64, 1}, 1}(undef, ncells^2)

srt_idx_tree[1] = [collect(1:natoms)]

#####################

d = 2
pl = 1
ej = 1

idx_up = arbol[d][pl][ej]
tre_up = srt_idx_tree[d][pl]
tre_at = Array{Int64, 1}

In [32]:
function spread(srt_xyz, nodes_ranges, nodes_bounds, xyz_min, natoms,
        cutoff, nlevels, d, dbg::Bool = false)
    
    nodes_indices_xyz = Array{Array{UnitRange{Int64}, 1}, 1}(undef, nlevels+1)
    nodes_indices_xyz[1] = [1:natoms]

    for i = 1:nlevels
        k = i + 1
        nodes_indices_xyz[k] = []
        for j = 1:2^(i-1)
            englobing_range = nodes_indices_xyz[i][j]
            ancho = nodes_ranges[k][j*2][2] - nodes_ranges[k][j*2-1][1]
            if ancho > 1
                if englobing_range.start == 0 & englobing_range.stop == 0
                    # 0 átomos en la celda
                    push!(nodes_indices_xyz[k], 0:0)
                    push!(nodes_indices_xyz[k], 0:0)
                elseif length(englobing_range) >= 1
                    xyz_idx = searchsortedlast(srt_xyz[englobing_range],
                        (xyz_min[d] + cutoff * nodes_bounds[k][j]))
                    
                    if xyz_idx == 0
                        # Todos los átomos en la celda de la derecha
                        push!(nodes_indices_xyz[k], 0:0)
                        push!(nodes_indices_xyz[k], englobing_range)
                    elseif xyz_idx == length(englobing_range)
                        # Todos los átomos en la celda de la izquierda
                        push!(nodes_indices_xyz[k], englobing_range)
                        push!(nodes_indices_xyz[k], 0:0)
                    else
                        # Divido en 2 celdas más
                        xyz_boundary = xyz_idx + englobing_range.start - 1
                        push!(nodes_indices_xyz[k],
                            nodes_indices_xyz[i][j].start:xyz_boundary)
                        push!(nodes_indices_xyz[k],
                            xyz_boundary+1:nodes_indices_xyz[i][j].stop)
                    end
                else
                    println(srt_xyz, xyz_min, "  ", natoms, "  ", englobing_range)
                    error("re loco.")
                end
            else
                push!(nodes_indices_xyz[k], englobing_range)
            end
        end
    end

    return nodes_indices_xyz[end]
end

spread (generic function with 2 methods)

In [340]:
arbol = contacto.tree(ncells, natoms, xyz);
nodo = arbol.root;

x_tree = spread(nodo.xyz[nodo.srt_idx, nodo.node_level+1], 
    nodes_ranges, nodes_bounds, xyz_min, nodo.natoms, cutoff,
    nlevels, nodo.node_level+1)

let i = 1
    for i = 1:ncells
        nodo[i].idx_range = x_tree[i]
        
        if nodo[i].idx_range == 0:0
            nodo[i].natoms = 0
            nodo[i].srt_idx = [0]
            nodo[i].xyz = [0. 0.]
            for j = 1:nodo.n
                for k = 1:nodo.n
                    nodo[i][j][k].idx_range == 0:0
                    nodo[i][j][k].natoms = 0
                    nodo[i][j][k].srt_idx = [0]
                    nodo[i][j][k].xyz = [0. 0. 0.]
                end
            end
        else
            nodo[i].natoms = length(nodo[i].idx_range)
            nodo[i].srt_idx = nodo.srt_idx[nodo[i].idx_range]
            nodo[i].xyz = nodo.xyz[nodo[i].srt_idx, :]
                
            srt_idx_y = sortperm(nodo[i].xyz[:, nodo[i].node_level+1])
            srt_y = nodo[i].xyz[srt_idx_y, nodo[i].node_level+1]
            y_tree = spread(srt_y, nodes_ranges, nodes_bounds, xyz_min,
                nodo[i].natoms, cutoff, nlevels, nodo[i].node_level+1, true)
            
            let j = 1
                for j = 1:nodo.n
                    nodo[i][j].idx_range = y_tree[j]
            
                    if nodo[i][j].idx_range == 0:0
                        nodo[i][j].natoms = 0
                        nodo[i][j].srt_idx = [0]
                        nodo[i][j].xyz = [0. 0. 0.]
                        for k = 1:nodo.n
                            nodo[i][j][k].idx_range = 0:0
                            nodo[i][j][k].natoms = 0
                            nodo[i][j][k].srt_idx = [0]
                            nodo[i][j][k].xyz = [0. 0. 0.]
                        end
                    else
                        nodo[i][j].natoms = length(nodo[i][j].idx_range)
                        nodo[i][j].srt_idx =
                            nodo[i].srt_idx[srt_idx_y[nodo[i][j].idx_range]]
                        nodo[i][j].xyz = nodo.xyz[nodo[i][j].srt_idx, :]
                        
                        srt_idx_z = sortperm(
                            nodo[i][j].xyz[:, nodo[i][j].node_level+1])
                        srt_z = nodo[i][j].xyz[srt_idx_z, nodo[i][j].node_level+1]
                        
                        z_tree = spread(srt_z, nodes_ranges, nodes_bounds,
                            xyz_min, nodo[i][j].natoms, cutoff, nlevels,
                            nodo[i][j].node_level+1, true)
                        let k = 1
                            for k = 1:nodo.n
                                nodo[i][j][k].idx_range = z_tree[k]
            
                                if nodo[i][j][k].idx_range == 0:0
                                    nodo[i][j][k].natoms = 0
                                    nodo[i][j][k].srt_idx = [0]
                                    nodo[i][j][k].xyz = [0. 0. 0.]
                                else
                                    nodo[i][j][k].natoms = 
                                        length(nodo[i][j][k].idx_range)
                                    nodo[i][j][k].srt_idx = 
                                        nodo[i].srt_idx[srt_idx_y[srt_idx_z[
                                            nodo[i][j][k].idx_range]]]
                                    nodo[i][j][k].xyz = 
                                        nodo.xyz[nodo[i][j][k].srt_idx, :]
                                end
                            end
                        end
                    end
                end
            end
        end
    end
end

┌ Warning: Deprecated syntax `implicit assignment to global variable `j``.
│ Use `global j` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `k``.
│ Use `global k` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `k``.
│ Use `global k` instead.
└ @ nothing none:0


In [217]:
nodes_bounds

3-element Array{Array{Int64,1},1}:
 [4]   
 [2]   
 [1, 3]

In [71]:
cell_list_mtx = Array{Array{Int64, 1}, 3}(undef, ncells, ncells, ncells)
for i = 1:ncells
    for j = 1:ncells
        for k = 1:ncells
            cell_list_mtx[i, j, k] = Array{Int64, 1}()
            cell_list_mtx[i, j, k] = nodo[i][j][k].srt_idx
        end
    end
end

In [81]:
println(cell_list_mtx[5, :, :])

Array{Int64,1}[[0] [303, 1167] [118, 1119, 364, 1118, 302, 329, 363, 1156] [1148, 1158, 400, 366, 359, 357, 1117] [1122, 1163, 1126, 356, 102] [0] [0] [0] [0] [0] [0]; [0] [303, 1167] [118, 1119, 364, 1118, 302, 329, 363] [1156, 1148, 1158, 400, 366, 359] [357, 1117, 1122, 1163, 1126, 356, 102, 375, 1165] [368, 408] [0] [0] [0] [0] [0]; [0] [303, 1167, 118, 1119, 364, 1118] [302, 329, 363, 1156, 1148, 1158, 400, 366, 359, 357, 1117, 1122, 1163, 1126, 356] [102, 375, 1165, 368, 408, 111, 330, 109, 113, 103, 403, 438, 437, 115, 75, 342, 327, 333, 358, 117, 1157, 412, 1168, 417] [101, 1166, 1123, 304, 82, 1169, 360, 399, 402, 114] [436, 343, 331, 371] [0] [0] [0] [0] [0]; [0] [0] [303, 1167, 118, 1119, 364, 1118, 302, 329, 363, 1156, 1148] [1158, 400, 366, 359, 357, 1117, 1122, 1163, 1126, 356, 102, 375, 1165] [368, 408, 111, 330, 109, 113, 103, 403, 438, 437, 115, 75, 342] [327, 333, 358, 117, 1157, 412, 1168, 417] [0] [0] [0] [0] [0]; [0] [303] [1167, 118, 1119, 364, 1118, 302] [329, 36

In [88]:
cell_list_mtx[11, 3, 2:3]

2-element Array{Array{Int64,1},1}:
 [0]                               
 [699, 1978, 1405, 733, 1381, 1406]

In [120]:
argmax(cell_list_mtx)

CartesianIndex(7, 3, 2)

In [318]:
nodo[3][1][6].xyz

1×3 Array{Float64,2}:
 0.0  0.0  0.0

In [316]:
12 + 61 + 19

92

In [301]:
inita = Array{Array{Float64,2}, 1}(undef, ncells)
i = 3
j = 1
k = 1
for a = 1:ncells
    inita[a] = nodo[i][j][k].xyz
    let j = 2
        for j = 2:ncells
            inita[a] = vcat(inita[a], nodo[i][j][k].xyz)
        end
    end
end

a = 1
ploteo = scatter(inita[a][:, 1], inita[a][:, 2], inita[a][:, 3],
    xlim = (x_min, x_max), ylim = (y_min, y_max), zlim = (z_min, z_max),
    xlabel = "X", ylabel = "Y", zlabel = "Z")

for a = 2:ncells
    ploteo = scatter!(inita[a][:, 1], inita[a][:, 2], inita[a][:, 3],
    xlim = (x_min, x_max), ylim = (y_min, y_max), zlim = (z_min, z_max),
    xlabel = "X", ylabel = "Y", zlabel = "Z")
end

┌ Warning: Deprecated syntax `implicit assignment to global variable `a``.
│ Use `global a` instead.
└ @ nothing none:0
┌ Warning: Loop variable `a` around In[301]:5 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[301]:5


In [302]:
ploteo

In [341]:
i = 3
j = 3
k = 1
ploteo = scatter(nodo[i][j][k].xyz[:, 1], nodo[i][j][k].xyz[:, 2],
    nodo[i][j][k].xyz[:, 3],
xlim = (x_min, x_max), ylim = (y_min, y_max), zlim = (z_min, z_max),
xlabel = "X", ylabel = "Y", zlabel = "Z")

for k = 2:ncells
    ploteo = scatter!(nodo[i][j][k].xyz[:, 1], nodo[i][j][k].xyz[:, 2],
        nodo[i][j][k].xyz[:, 3],
        xlim = (x_min, x_max), ylim = (y_min, y_max), zlim = (z_min, z_max),
        xlabel = "X", ylabel = "Y", zlabel = "Z")
end
ploteo

┌ Warning: Deprecated syntax `implicit assignment to global variable `k``.
│ Use `global k` instead.
└ @ nothing none:0
┌ Warning: Loop variable `k` around In[341]:9 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[341]:9


In [275]:
i = 3
j = 1
ploteo = scatter(nodo[i][j].xyz[:, 1], nodo[i][j].xyz[:, 2], nodo[i][j].xyz[:, 3],
xlim = (x_min, x_max), ylim = (y_min, y_max), zlim = (z_min, z_max),
xlabel = "X", ylabel = "Y", zlabel = "Z")

for i = 3:3
    for j = 2:ncells
        ploteo = scatter!(nodo[i][j].xyz[:, 1], nodo[i][j].xyz[:, 2],
                                        nodo[i][j].xyz[:, 3],
            xlim = (x_min, x_max), ylim = (y_min, y_max), zlim = (z_min, z_max),
            xlabel = "X", ylabel = "Y", zlabel = "Z")
    end
end

┌ Warning: Deprecated syntax `implicit assignment to global variable `i``.
│ Use `global i` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `j``.
│ Use `global j` instead.
└ @ nothing none:0
┌ Warning: Loop variable `i` around In[275]:7 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[275]:7
┌ Warning: Loop variable `j` around In[275]:8 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[275]:8


In [276]:
ploteo

In [279]:
i = 3
ploteo = scatter(nodo[i].xyz[:, 1], nodo[i].xyz[:, 2], nodo[i].xyz[:, 3],
xlim = (x_min, x_max), ylim = (y_min, y_max), zlim = (z_min, z_max),
xlabel = "X", ylabel = "Y", zlabel = "Z")

for i = 3:3
        ploteo = scatter!(nodo[i].xyz[:, 1], nodo[i].xyz[:, 2],
                                        nodo[i].xyz[:, 3],
            xlim = (x_min, x_max), ylim = (y_min, y_max), zlim = (z_min, z_max),
            xlabel = "X", ylabel = "Y", zlabel = "Z")
end

┌ Warning: Deprecated syntax `implicit assignment to global variable `i``.
│ Use `global i` instead.
└ @ nothing none:0
┌ Warning: Loop variable `i` around In[279]:6 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[279]:6


In [280]:
ploteo

In [239]:
nodo[1][1].xyz

13×3 Array{Float64,2}:
 -37.345  23.789  -32.38 
 -36.838  22.714  -33.699
 -36.731  23.087  -32.766
 -36.295  23.647  -30.474
 -35.997  22.375  -31.997
 -35.955  22.722  -30.694
 -35.342  22.357  -29.979
 -35.202  21.443  -32.49 
 -35.194  21.324  -33.493
 -34.515  20.57   -30.768
 -34.325  19.536  -32.113
 -34.259  20.586  -31.828
 -32.798  20.912  -32.241

In [88]:
idx_xy = findall(x -> x == tmp_srt_idx_t_xyz_y[lado_y][i],
            tmp_srt_idx_t_xyz_x[lado_x])

1-element Array{Int64,1}:
 27

In [ ]:
y_md_bound = searchsortedlast(srt_y_t_xyz[nodes_indices[i][j]],
    (y_min + cutoff * nodes_bounds[k][j]))
z_md_bound = searchsortedlast(srt_z_t_xyz[nodes_indices[i][j]],
    (z_min + cutoff * nodes_bounds[k][j]))

In [100]:
lado_y = 1
for i = 1:length(tmp_srt_idx_t_xyz_y[lado_y])
    lado_x = 1
    idx_xy = findall(x -> x == tmp_srt_idx_t_xyz_y[lado_y][i],
            tmp_srt_idx_t_xyz_x[lado_x])
    
    lado_z = 1
    if isempty(idx_xy)
    # está en -x, -y, 
        idx_zy = findall(z -> z == tmp_srt_idx_t_xyz_y[lado_y][i],
            tmp_srt_idx_t_xyz_z[lado_z])
        
        if isempty(idx_yz)
        # está en -x, -y, -z
            
        else
            
        end
    
        
    else
        
        if  srt_idx_t_xyz[x_lh_bound:x_rh_bound, 1][idx_xy] ==
        srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i]
        
            lado_z = 1
                
            idx_zy = searchsortedlast(srt_idx_t_xyz[z_lh_bound:z_rh_bound, 3],
                srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i])
            if idx_zy != 0
                if  srt_idx_t_xyz[z_lh_bound:z_rh_bound, 3][idx_zy] ==
                srt_idx_t_xyz[y_lh_bound:y_rh_bound, 2][i]
                
                    push!(octant_0, i)
                end
            end
        end
    end
end

4-element Array{Array{Int64,1},1}:
 [11]                    
 [6, 5]                  
 [3, 3, 3, 2]            
 [2, 1, 2, 1, 2, 1, 1, 1]